# PaDiM on SageMaker--推理

## 运行环境
Kernel 选择pytorch_latest_p37。  
本文在boto3 1.17.12和sagemaker 2.26.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

## 本地虚拟环境推理(可选)
新启动一个shell窗口  
```
sudo mkdir /opt/ml  
sudo chmod 777 /opt/ml  
conda activate pytorch_latest_p37
```
必须cd到`2-inference/source`目录，然后运行`python predictor.py`，正常启动会输出以下内容：
```
 * Serving Flask app "predictor" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```

In [ ]:
#修改请求图片
#如果仅请求一个图片会报错
#upload_bucket、upload_path是推理完毕后，结果上传bucket名称和路径
#bucket、image_uri是待推理的bucket名称和图片
!curl -H "Content-Type: application/json" -X POST --data '{"upload_bucket":"junzhong","upload_path":"result/ad2","bucket":"junzhong","image_uri":["data/mvtec/bottle/test/broken_small/000.png","data/mvtec/bottle/test/broken_small/001.png"]}' http://127.0.0.1:5000/invocations

## Amazon 深度学习容器

* [容器镜像清单](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)
* 本文基于pytorch inference: `727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-inference:1.6.0-gpu-py36-cu101-ubuntu16.04`

## Build自定义推理镜像

In [ ]:
import boto3
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'ad-inference'
tag = 'latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + ":" + tag)
print(image_uri)
ecr = '{}.dkr.ecr.{}.{}'.format(account_id, region, uri_suffix)

In [ ]:
#国内pytorch inference基础镜像地址，不要修改
base_img='727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-inference:1.6.0-gpu-py36-cu101-ubuntu16.04'
#登录基础镜像ECR，不要修改
!aws ecr get-login-password --region cn-northwest-1 | docker login --username AWS --password-stdin 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
%%time
!docker build -t $ecr_repository -f Dockerfile --build-arg BASE_IMG=$base_img .

### 本地容器推理(可选)

In [ ]:
!docker run -p 8080:8080 -d --rm $ecr_repository:$tag serve

In [ ]:
#修改请求图片
#如果仅请求一个图片会报错
#upload_bucket、upload_path是推理完毕后，结果上传bucket名称和路径
#bucket、image_uri是待推理的bucket名称和图片
!curl -H "Content-Type: application/json" -X POST --data '{"upload_bucket":"junzhong","upload_path":"result/ad","bucket":"junzhong","image_uri":["data/mvtec/bottle/test/broken_small/000.png","data/mvtec/bottle/test/broken_small/001.png"]}' http://127.0.0.1:8080/invocations

### 推送容器image到ECR

In [ ]:
!docker tag $ecr_repository:$tag $image_uri
!$(aws ecr get-login --no-include-email)
!docker push $image_uri

## 部署模型到SageMaker

In [ ]:
from sagemaker.model import Model
import boto3

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题
print(role)

endpoint_name = "ad"
my_model = Model(
            role=role,
            image_uri=image_uri)

#该步骤大概需要15分钟
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.m5.xlarge'
                                )

## 推理

### 准备测试图片

In [ ]:
#修改请求图片
#如果仅请求一个图片会报错
#upload_bucket、upload_path是推理完毕后，结果上传bucket名称和路径
#bucket、image_uri是待推理的bucket名称和图片
data = {"upload_bucket":"junzhong","upload_path":"result/ad","bucket":"junzhong","image_uri":["data/mvtec/bottle/test/broken_small/000.png","data/mvtec/bottle/test/broken_small/001.png"]}

### 方式一、使用boto3

In [ ]:
import boto3
import json
endpoint_name = "ad"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data),
                                  ContentType="application/json")
#print(response)
response_body = response['Body'] 
body= response_body.read()
result = json.loads(body.decode('utf-8'))
print(result)

### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
result = predictor.predict(data)
print(result)

## 使用结果

### 下载原图并显示

In [ ]:
from IPython.display import Image
s3_file="s3://"+data["upload_bucket"]+"/"+data["upload_path"]
!aws s3 sync $s3_file ./result

In [ ]:
Image(filename="result/0.png", width=600)

## 删除Endpoint

In [ ]:
import boto3
sage = boto3.Session().client(service_name='sagemaker') 
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_name)